# Homework 2

## Exercise 1
Assume we have 1 dollar to be invested in two assets, whose return is modeled as a bivariate Gaussian distribution such that

$$
\Sigma = \begin{pmatrix}
0.25 & 0.15 \\ 0.15 & 0.10
\end{pmatrix}
$$

How much should we invest in each to minimize the overall variance in return with and without short selling? Answer this question solving the problem analytically and using a numerical code for constrained optimization problems. To simplify the problem, the acceptable baseline expected rate of return $m_b$ is $−\infty$.

### Analytical solution
In this problem we have to minimize the variance with inequality constrains. This variance is computed in the following way $f(\textbf{w}) = \textbf{w}^\top \Sigma\textbf{w}$ where each component of the vector $\textbf{w}$ is the weight that the corresponding asset is going to have in our portfolio.

#### With short selling

The constrains that we have to take into account are $\sum_i^nw_i=1$, this means that our weights must sum to 1. Now to solve the problem lets compute the function $f(\textbf{w})$.

$$
    f(\textbf{w}) = \begin{pmatrix} w_1 & w_2 \end{pmatrix} \begin{pmatrix} 0.25 & 0.15 \\ 0.15 & 0.10 \end{pmatrix}\begin{pmatrix} w_1 \\ w_2 \end{pmatrix}= 0.25 w_1^2+0.3w_1w_2+0.1w_2^2=f(w_1,w_2).
$$

With this function we have to solve a optimization problem in two dimensions but we can transform it by taking into account the constrain $w_1+w_2=1\to w_1=1-w_2$. So if we substitute this in the original equation we get $f(w_2) = 0.05w_2^2-0.2w_2+0.25,$ that is a function of only one variable so to find its minimum we just have to derive it.

$$
f'(w_2) = 0.1w_2-0.2.
$$

And the minimum is achieved at $0.1w_2 = 0.2\to w_2 = 2$. With this value of $w_2$ we get $w_1=-1$ and the solution is:

$$
    \textbf{w} = \begin{pmatrix}-1\\2\end{pmatrix}.
$$

With this solution we see that we have to short sell the first asset and then buy the second one.

#### Without short selling
In this case we don't allow the weights to be negative, but the rest of the problem is the same. We have to implement the following constrains $w_1\geq0$ and $w_2\geq0$. Because $w_2$ is positive and $w_1=1-w_2$ is also positive we have that $w_2$ must be also less or equal than 1, so $w_2\in[0,1]$. The absolute minimum from of the function $f(w_2)$, obtained in [the previous section](#with-short-selling), is not inside this region so because we know that this function only has an extrema point the minimum value inside the interval must be achieved at the boundaries.

$$
f(0) = 0.25 \;\;\;\text{and}\;\;\; f(1) = 0.1
$$

So the minimum is at $w_2=1$ therefore $w_1$ must be 0.

$$
    \textbf{w} = \begin{pmatrix}0\\1\end{pmatrix}.
$$

## Numeric solution

In [2]:
0.2/0.1

2.0